<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepNLP-END2.0/blob/main/05_NLP_Augment/SSTModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Thu Jun  3 19:44:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install pytorch-lightning --quiet
! pip install OmegaConf --quiet
! pip install nlpaug --quiet
! pip install gdown==3.13.0

     |████████████████████████████████| 808kB 26.6MB/s 
     |████████████████████████████████| 276kB 52.6MB/s 
     |████████████████████████████████| 10.6MB 36.1MB/s 
     |████████████████████████████████| 645kB 50.8MB/s 
     |████████████████████████████████| 112kB 57.8MB/s 
     |████████████████████████████████| 829kB 42.8MB/s 
     |████████████████████████████████| 1.3MB 50.1MB/s 
     |████████████████████████████████| 296kB 41.5MB/s 
     |████████████████████████████████| 143kB 59.2MB/s 
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.
     |████████████████████████████████| 399kB 30.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.13.0-cp37-none-any.whl size=9046 sha256=b56b77ab579a096929f29a279815d92a0520dfe5f9228beeed54760c62092396
  Stored in directory: /root/.cache/pip/wheels/ba

In [14]:
import copy

import torch
import torchtext
import pytorch_lightning as pl

from torch import nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from pytorch_lightning.metrics.functional import accuracy

from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.experimental.functional import sequential_transforms, ngrams_func, totensor, vocab_func
from torchtext.vocab import build_vocab_from_iterator

import torchtext.experimental.functional as text_f

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

import random
import gdown

import pandas as pd
import numpy as np

from tqdm.auto import tqdm
from pathlib import Path
from omegaconf import OmegaConf
from zipfile import ZipFile

from typing import Optional, Tuple, Any, Dict, List

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
plt.style.use("dark_background")

In [15]:
class StanfordSentimentTreeBank(Dataset):
    """The Standford Sentiment Tree Bank Dataset
    Stanford Sentiment Treebank V1.0

    This is the dataset of the paper:

    Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank
    Richard Socher, Alex Perelygin, Jean Wu, Jason Chuang, Christopher Manning, Andrew Ng and Christopher Potts
    Conference on Empirical Methods in Natural Language Processing (EMNLP 2013)

    If you use this dataset in your research, please cite the above paper.

    @incollection{SocherEtAl2013:RNTN,
    title = {{Parsing With Compositional Vector Grammars}},
    author = {Richard Socher and Alex Perelygin and Jean Wu and Jason Chuang and Christopher Manning and Andrew Ng and Christopher Potts},
    booktitle = {{EMNLP}},
    year = {2013}
    }
    """

    ORIG_URL = "http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip"
    DATASET_NAME = "StanfordSentimentTreeBank"
    URL = 'https://drive.google.com/uc?id=1urNi0Rtp9XkvkxxeKytjl1WoYNYUEoPI'
    OUTPUT = 'sst_dataset.zip'
 

    def __init__(self, root, vocab=None, text_transforms=None, label_transforms=None, split='train', ngrams=1, use_transformed_dataset=True):
        """Initiate text-classification dataset.
        Args:
            data: a list of label and text tring tuple. label is an integer.
                [(label1, text1), (label2, text2), (label2, text3)]
            vocab: Vocabulary object used for dataset.
            transforms: a tuple of label and text string transforms.
        """

        super(self.__class__, self).__init__()

        if split not in ['train', 'test']:
            raise ValueError(f'split must be either ["train", "test"] unknown split {split}')

        self.vocab = vocab

        gdown.cached_download(self.URL, Path(root) / self.OUTPUT)

        self.generate_sst_dataset(split, Path(root) / self.OUTPUT)

        tokenizer = get_tokenizer("basic_english")

        # the text transform can only work at the sentence level
        # the rest of tokenization and vocab is done by this class
        self.text_transform = sequential_transforms(tokenizer, text_f.ngrams_func(ngrams))

        def build_vocab(data, transforms):
            def apply_transforms(data):
                for line in data:
                    yield transforms(line)
            return build_vocab_from_iterator(apply_transforms(data), len(data))

        if self.vocab is None:
            # vocab is always built on the train dataset
            self.vocab = build_vocab(self.dataset_train["phrase"], self.text_transform)


        if text_transforms is not None:
            self.text_transform = sequential_transforms(
                self.text_transform, text_transforms, text_f.vocab_func(self.vocab), text_f.totensor(dtype=torch.long)
            )
        else:
            self.text_transform = sequential_transforms(
                self.text_transform, text_f.vocab_func(self.vocab), text_f.totensor(dtype=torch.long)
            )

        self.label_transform = sequential_transforms(text_f.totensor(dtype=torch.long))

    def generate_sst_dataset(self, split, dataset_file):

        with ZipFile(dataset_file) as datasetzip:
            with datasetzip.open('sst_dataset/sst_dataset_augmented.csv') as f:
                dataset = pd.read_csv(f, index_col=0)

        self.dataset_orig = dataset.copy()

        dataset_train_raw = dataset[dataset['splitset_label'].isin([1, 3])]
        self.dataset_train = pd.concat([
                dataset_train_raw[['phrase_cleaned', 'sentiment_values']].rename(columns={"phrase_cleaned": 'phrase'}),
                dataset_train_raw[['synonym_sentences', 'sentiment_values']].rename(columns={"synonym_sentences": 'phrase'}),
                dataset_train_raw[['backtranslated', 'sentiment_values']].rename(columns={"backtranslated": 'phrase'}),
        ], ignore_index=True)

        if split == 'train':
            self.dataset = self.dataset_train.copy()
        else:
            self.dataset = dataset[dataset['splitset_label'].isin([2])] \
                                    [['phrase_cleaned', 'sentiment_values']] \
                                    .rename(columns={"phrase_cleaned": 'phrase'}) \
                                    .reset_index(drop=True)

    @staticmethod
    def discretize_label(label):
        if label <= 0.2: return 0
        if label <= 0.4: return 1
        if label <= 0.6: return 2
        if label <= 0.8: return 3
        return 4

    def __getitem__(self, idx):
        # print(f'text: {self.dataset["sentence"].iloc[idx]}, label: {self.dataset["sentiment_values"].iloc[idx]}')
        text = self.text_transform(self.dataset['phrase'].iloc[idx])
        label = self.label_transform(self.dataset['sentiment_values'].iloc[idx])
        # print(f't_text: {text} {text.shape}, t_label: {label}')
        return label, text 

    def __len__(self):
        return len(self.dataset)

    @staticmethod
    def get_labels():
        return ['very negative', 'negative', 'neutral', 'positive', 'very positive']

    def get_vocab(self):
        return self.vocab

    @property
    def collator_fn(self):
        def collate_fn(batch):
            pad_idx = self.get_vocab()['<pad>']
            
            labels, sequences = zip(*batch)

            labels = torch.stack(labels)

            lengths = torch.LongTensor([len(sequence) for sequence in sequences])

            # print('before padding: ', sequences[40])
            
            sequences = torch.nn.utils.rnn.pad_sequence(sequences, 
                                                        padding_value = pad_idx,
                                                        batch_first=True
                                                        )
            # print('after padding: ', sequences[40])
                    
            return labels, sequences, lengths
        
        return collate_fn

In [16]:
class SSTDataModule(pl.LightningDataModule):
    """
    DataModule for SST, train, val, test splits and transforms
    """

    name = "stanford_sentiment_treebank"

    def __init__(
        self,
        data_dir: str = '.',
        val_split: int = 1000,
        num_workers: int = 2,
        batch_size: int = 64,
        *args,
        **kwargs,
    ):
        """
        Args:
            data_dir: where to save/load the data
            val_split: how many of the training images to use for the validation split
            num_workers: how many workers to use for loading data
            normalize: If true applies image normalize
            batch_size: desired batch size.
        """
        super().__init__(*args, **kwargs)

        self.data_dir = data_dir
        self.val_split = val_split
        self.num_workers = num_workers
        self.batch_size = batch_size

        self.dataset_train = ...
        self.dataset_val = ...
        self.dataset_test = ...

        self.SST = StanfordSentimentTreeBank

    def prepare_data(self):
        """Saves IMDB files to `data_dir`"""
        self.SST(self.data_dir)

    def setup(self, stage: Optional[str] = None):
        """Split the train and valid dataset"""

        train_trans, test_trans = self.default_transforms

        train_dataset = self.SST(self.data_dir, split='train', **train_trans)
        test_dataset = self.SST(self.data_dir, split='test', **test_trans)

        train_length = len(train_dataset)

        self.raw_dataset_train = train_dataset
        self.raw_dataset_test = test_dataset

        # self.dataset_train, self.dataset_val = random_split(train_dataset, [train_length - self.val_split, self.val_split])
        self.dataset_train = train_dataset
        self.dataset_test = test_dataset

    def train_dataloader(self):
        """IMDB train set removes a subset to use for validation"""
        loader = DataLoader(
            self.dataset_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
            collate_fn=self.collator_fn
        )
        return loader

    def val_dataloader(self):
        """IMDB val set uses a subset of the training set for validation"""
        loader = DataLoader(
            self.dataset_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
            collate_fn=self.collator_fn
        )
        return loader

    def test_dataloader(self):
        """IMDB test set uses the test split"""
        loader = DataLoader(
            self.dataset_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
            collate_fn=self.collator_fn
        )
        return loader

    def get_vocab(self):
        return self.raw_dataset_train.get_vocab()

    @property
    def default_transforms(self):
        train_transforms = {
            'text_transforms': text_f.sequential_transforms(
                random_deletion,
                random_swap
            ),
            'label_transforms': None
        }
        test_transforms = {
            'text_transforms': None,
            'label_transforms': None
        }

        return train_transforms, test_transforms

    @property
    def collator_fn(self):
        return self.raw_dataset_train.collator_fn

In [17]:
def random_deletion(words, p=0.1): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0, 1) > p, words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

def random_swap(sentence, n=3, p=0.1): 
    length = range(len(sentence))
    n = min(n, len(sentence))
    for _ in range(n):
        if random.uniform(0, 1) > p:
            idx1, idx2 = random.choices(length, k=2)
            sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [18]:
class SSTModel(pl.LightningModule):

    def __init__(self, hparams, *args, **kwargs):
        super().__init__()

        self.save_hyperparameters(hparams)

        self.num_classes = self.hparams.output_dim

        self.embedding = nn.Embedding(self.hparams.input_dim, self.hparams.embedding_dim)

        self.lstm = nn.LSTM(
            self.hparams.embedding_dim, 
            self.hparams.hidden_dim, 
            num_layers=self.hparams.num_layers,
            dropout=self.hparams.dropout,
            batch_first=True
        )

        self.proj_layer = nn.Sequential(
            nn.Linear(self.hparams.hidden_dim, self.hparams.hidden_dim),
            nn.BatchNorm1d(self.hparams.hidden_dim),
            nn.ReLU(),
            nn.Dropout(self.hparams.dropout),
        )

        self.fc = nn.Linear(self.hparams.hidden_dim, self.num_classes)

        self.loss = nn.CrossEntropyLoss()

    def init_state(self, sequence_length):
        return (torch.zeros(self.hparams.num_layers, sequence_length, self.hparams.hidden_dim).to(self.device),
                torch.zeros(self.hparams.num_layers, sequence_length, self.hparams.hidden_dim).to(self.device))

    def forward(self, text, text_length, prev_state=None):
        # [batch size, sentence length] => [batch size, sentence len, embedding size]
        embedded = self.embedding(text)

        # packs the input for faster forward pass in RNN
        packed = torch.nn.utils.rnn.pack_padded_sequence(
            embedded, text_length.to('cpu'), 
            enforce_sorted=False, 
            batch_first=True
        )
        
        # [batch size sentence len, embedding size] => 
        #   output: [batch size, sentence len, hidden size]
        #   hidden: [batch size, 1, hidden size]
        packed_output, curr_state = self.lstm(packed, prev_state)

        hidden_state, cell_state = curr_state

        # print('hidden state shape: ', hidden_state.shape)
        # print('cell')

        # unpack packed sequence
        # unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

        # print('unpacked: ', unpacked.shape)

        # [batch size, sentence len, hidden size] => [batch size, num classes]
        # output = self.proj_layer(unpacked[:, -1])
        output = self.proj_layer(hidden_state[-1])

        # print('output shape: ', output.shape)

        output = self.fc(output)

        return output, curr_state

    def shared_step(self, batch, batch_idx):
        label, text, text_length = batch

        logits, in_state = self(text, text_length)
        
        loss = self.loss(logits, label)

        pred = torch.argmax(F.log_softmax(logits, dim=1), dim=1)
        acc = accuracy(pred, label)

        metric = {'loss': loss, 'acc': acc} 
        
        return metric


    def training_step(self, batch, batch_idx):
        metrics = self.shared_step(batch, batch_idx)

        log_metrics = {'train_loss': metrics['loss'], 'train_acc': metrics['acc']}

        self.log_dict(log_metrics, prog_bar=True)

        return metrics


    def validation_step(self, batch, batch_idx):
        metrics = self.shared_step(batch, batch_idx)

        return metrics
    

    def validation_epoch_end(self, outputs):
        acc = torch.stack([x['acc'] for x in outputs]).mean()
        loss = torch.stack([x['loss'] for x in outputs]).mean()

        log_metrics = {'val_loss': loss, 'val_acc': acc}

        self.log_dict(log_metrics, prog_bar=True)

        return log_metrics


    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def test_epoch_end(self, outputs):
        accuracy = torch.stack([x['acc'] for x in outputs]).mean()

        self.log('hp_metric', accuracy)

        self.log_dict({'test_acc': accuracy}, prog_bar=True)


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        lr_scheduler = {
            'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, verbose=True),
            'monitor': 'train_loss',
            'name': 'scheduler'
        }
        return [optimizer], [lr_scheduler]


**Sanity Checking**



In [19]:
sst_dataset = SSTDataModule(batch_size=128)
sst_dataset.setup()

  0%|          | 0/27483 [00:00<?, ?lines/s]

File exists: sst_dataset.zip


  0%|          | 0/27483 [00:00<?, ?lines/s]

File exists: sst_dataset.zip


100%|██████████| 27483/27483 [00:00<00:00, 57072.81lines/s]


In [20]:
loader = sst_dataset.train_dataloader()

In [21]:
batch = next(iter(loader))

In [22]:
label, text, text_length = batch

In [23]:
text.size(0)

128

In [24]:
label.shape, text.shape, text_length.shape

(torch.Size([128]), torch.Size([128, 41]), torch.Size([128]))

In [25]:
text[0]

tensor([    2,   224,     6, 21679,  6333,   513,  1097,   175,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1])

In [186]:
hparams = OmegaConf.create({
    'input_dim': len(sst_dataset.get_vocab()),
    'embedding_dim': 128,
    'num_layers': 2,
    'hidden_dim': 64,
    'dropout': 0.5,
    'output_dim': len(StanfordSentimentTreeBank.get_labels()),
    'lr': 5e-4,
    'epochs': 30,
    'use_lr_finder': False
})

In [187]:
sst_model = SSTModel(hparams)

In [188]:
output, (h, c) = sst_model(text, text_length)

In [189]:
output.shape

torch.Size([128, 5])

In [190]:
sst_model = SSTModel(hparams)

In [191]:
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=3,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

In [192]:
trainer = pl.Trainer(gpus=1, max_epochs=hparams.epochs, callbacks=[lr_monitor, checkpoint_callback], progress_bar_refresh_rate=1, reload_dataloaders_every_epoch=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [193]:
if hparams.use_lr_finder:
    # Run learning rate finder
    lr_finder = trainer.tuner.lr_find(sst_model, sst_dataset, max_lr=5)

    # Plot with
    fig = lr_finder.plot(suggest=True)
    fig.show()

    # Pick point based on plot, or get suggestion
    new_lr = lr_finder.suggestion()

    print(f'lr finder suggested lr: {new_lr}')

    # update hparams of the model
    sst_model.hparams.lr = new_lr

In [194]:
trainer.fit(sst_model, sst_dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | embedding  | Embedding        | 3.1 M 
1 | lstm       | LSTM             | 82.9 K
2 | proj_layer | Sequential       | 4.3 K 
3 | fc         | Linear           | 325   
4 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.784    Total estimated model params size (MB)


Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [195]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'hp_metric': 0.40006089210510254, 'test_acc': 0.40006089210510254}
--------------------------------------------------------------------------------


[{'hp_metric': 0.40006089210510254, 'test_acc': 0.40006089210510254}]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

## Model Diagnosis

In [155]:
loader = sst_dataset.test_dataloader()
batch = next(iter(loader))

In [157]:
label, text, text_length = batch
label.shape, text.shape, text_length.shape

(torch.Size([128]), torch.Size([128, 42]), torch.Size([128]))

In [182]:
def k_missclassified(batch, model, datamodule, k=10):
    model.eval()
    with torch.no_grad():
        label, text, text_length = batch

        logits, in_state = model(text, text_length)
    
        pred = torch.argmax(F.log_softmax(logits, dim=1), dim=1)
        
        acc = accuracy(pred, label)

    miss_idx = pred != label

    vocab = datamodule.get_vocab()
    for t, l, p in zip(text.numpy()[miss_idx][:k], label.numpy()[miss_idx][:k], pred.numpy()[miss_idx][:k]):
        sentence = ' '.join(vocab.itos[x] for x in t).replace(" <pad>", "")
        print('sentence: ', sentence)
        print(f'label: {datamodule.dataset_train.get_labels()[l]}, predicted: {datamodule.dataset_train.get_labels()[p]}')
        print('\n')

In [183]:
k_missclassified(batch, sst_model, sst_dataset)

sentence:  the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .
label: neutral, predicted: positive


sentence:  offers that rare combination of entertainment and education .
label: very positive, predicted: positive


sentence:  perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .
label: positive, predicted: neutral


sentence:  steers turns in a snappy screenplay that <unk> at the edges it ' s so clever you want to hate it .
label: positive, predicted: neutral


sentence:  but he somehow pulls it off .
label: positive, predicted: neutral


sentence:  take care of my cat offers a refreshingly different slice of asian cinema .
label: positive, predicted: very positive


sentence:  ultimately , it <unk> the reasons we need stories so much .
label: neutral, predicted: negative


sentence:  the movie ' s ripe , <unk> beauty will tempt those willing 

In [184]:
def k_correctclassified(batch, model, datamodule, k=10):
    model.eval()
    with torch.no_grad():
        label, text, text_length = batch

        logits, in_state = model(text, text_length)
    
        pred = torch.argmax(F.log_softmax(logits, dim=1), dim=1)
        
        acc = accuracy(pred, label)

    miss_idx = label == pred

    vocab = datamodule.get_vocab()
    for t, l, p in zip(text.numpy()[miss_idx][:k], label.numpy()[miss_idx][:k], pred.numpy()[miss_idx][:k]):
        sentence = ' '.join(vocab.itos[x] for x in t).replace(" <pad>", "")
        print('sentence: ', sentence)
        print(f'label: {datamodule.dataset_train.get_labels()[l]}, predicted: {datamodule.dataset_train.get_labels()[p]}')
        print('\n')

In [185]:
k_correctclassified(batch, sst_model, sst_dataset)

sentence:  effective but <unk> biopic
label: neutral, predicted: neutral


sentence:  if you sometimes like to go to the movies to have fun , wasabi is a good place to start .
label: positive, predicted: positive


sentence:  emerges as something rare , an issue movie that ' s so honest and keenly observed that it doesn ' t feel like one .
label: very positive, predicted: very positive


sentence:  this is a film well worth seeing , talking and singing heads and all .
label: very positive, predicted: very positive


sentence:  what really surprises about wisegirls is its low-key quality and genuine tenderness .
label: positive, predicted: positive


sentence:  <unk> wendigo is <unk> why we go to the cinema to be fed through the eye , the heart , the mind .
label: positive, predicted: positive


sentence:  one of the greatest family-oriented , fantasy-adventure movies ever .
label: very positive, predicted: very positive


sentence:  an utterly compelling ` who wrote it ' in which the r

## Misc Stuff

In [47]:
! ls

lightning_logs	sample_data  sst_dataset.zip


In [ ]:
ls lightning_logs/version_0

checkpoints/
events.out.tfevents.1622663662.f8a649598365.58.0
events.out.tfevents.1622664824.f8a649598365.58.1
hparams.yaml


In [45]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [46]:
! ls /gdrive/MyDrive/END2.0/05_NLP_Augment/

lightning_logs		   sst_dataset_translated.csv
sst_dataset_augmented.csv  sst_dataset_translated_gsheet.gsheet
sst_dataset_cleaned.csv    sst_dataset.zip
SST_Dataset.ipynb	   SSTModel.ipynb
sst_dataset_synonym.csv


In [49]:
# ! cp -r /gdrive/MyDrive/END2.0/05_NLP_Augment/lightning_logs .

In [136]:
# ! cp -r lightning_logs /gdrive/MyDrive/END2.0/05_NLP_Augment/

In [137]:
# drive.flush_and_unmount()

In [143]:
# ! rm -r lightning_logs

In [140]:
# ! du -sh *

838M	lightning_logs
55M	sample_data
4.9M	sst_dataset.zip


In [142]:
# ! tensorboard dev upload --logdir lightning_logs \
#     --name "END2 05_NLP_Augment - Satyajit" \
#     --description "Experiments on NLP Augmentation on SST Dataset"


2021-06-03 20:54:31.505467: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

lightning_logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%